In [1]:
import pandas as pd
df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\haberman-clustering.csv',encoding='unicode_escape')
df

,Age,Op_year,axil_nodes
0,30,62,3
1,30,65,0
2,31,59,2
3,31,65,4
4,33,58,10
...,...,...,...
277,75,62,1
278,76,67,0
279,77,65,3
280,78,65,1


In [2]:
from sklearn.model_selection import train_test_split
## we will do the splitting using a random state to ensure same splitting every time
X_train, X_test = train_test_split(df.values,test_size = .3,random_state = 13)

In [3]:
X_train

array([[52, 62,  0],
       [50, 66,  1],
       [49, 67,  1],
       [51, 64,  7],
       [47, 63,  6],
       [45, 65,  6],
       [54, 65, 23],
       [55, 66, 18],
       [70, 59,  8],
       [57, 62,  0],
       [54, 67, 46],
       [40, 65,  0],
       [49, 61,  1],
       [53, 58,  1],
       [58, 67,  0],
       [57, 67,  0],
       [47, 66,  0],
       [45, 59, 14],
       [55, 63,  6],
       [70, 63,  0],
       [56, 66,  1],
       [34, 61, 10],
       [64, 58,  0],
       [40, 58,  0],
       [35, 64, 13],
       [39, 63,  0],
       [36, 60,  1],
       [62, 65, 19],
       [55, 68, 15],
       [41, 60, 23],
       [38, 66, 11],
       [48, 67,  7],
       [66, 58,  0],
       [70, 58,  0],
       [57, 62, 14],
       [43, 64,  3],
       [52, 69,  3],
       [41, 58,  0],
       [43, 63, 14],
       [37, 60,  0],
       [56, 65,  9],
       [55, 58,  1],
       [45, 60,  0],
       [37, 58,  0],
       [60, 65,  0],
       [50, 63, 13],
       [61, 65,  8],
       [47, 6

In [4]:
X_test

array([[57, 64,  9],
       [71, 68,  2],
       [49, 60,  1],
       [45, 68,  0],
       [63, 61,  0],
       [59, 62, 35],
       [44, 58,  9],
       [50, 61,  0],
       [52, 63,  4],
       [51, 59,  1],
       [34, 60,  1],
       [42, 60,  1],
       [54, 68,  7],
       [58, 58,  0],
       [62, 66,  0],
       [53, 60,  1],
       [63, 61, 28],
       [45, 64,  0],
       [57, 61,  5],
       [50, 61,  6],
       [70, 66, 14],
       [34, 60,  0],
       [60, 61,  1],
       [62, 62,  6],
       [72, 67,  3],
       [41, 69,  8],
       [47, 68,  4],
       [34, 66,  9],
       [72, 64,  0],
       [51, 59, 13],
       [38, 67,  5],
       [42, 62, 20],
       [57, 64,  1],
       [39, 58,  0],
       [52, 64,  0],
       [37, 63,  0],
       [69, 66,  0],
       [74, 65,  3],
       [63, 61,  9],
       [55, 69,  3],
       [54, 66,  0],
       [46, 69,  3],
       [73, 62,  0],
       [63, 60,  1],
       [53, 58,  4],
       [65, 58,  0],
       [52, 65,  0],
       [73, 6

In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tensor_train = torch.FloatTensor(X_train).to(device)
tensor_test=torch.FloatTensor(X_test).to(device)

In [6]:
from autoencoder import Autoencoder
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
inputs_dim=df.shape[1]
n_bottleneck=10
model=Autoencoder(inputs_dim,n_bottleneck).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train_loader = DataLoader(TensorDataset(tensor_train, tensor_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(tensor_test, tensor_test), batch_size=64, shuffle=False)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from tqdm import tqdm
import os

lowest_val_loss = float('inf')  # Initialize with a large value
best_epoch = 0

num_epochs = 1000
for epoch in range(num_epochs):
    # Training
    model.train()
    total_train_loss = 0.0
    with tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Training', unit='batch') as t:
        for batch in t:
            input_data, _ = batch
            input_data = input_data.to(device)

            optimizer.zero_grad()
            output_data = model(input_data)
            loss = criterion(output_data, input_data)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            t.set_postfix(loss=f'{total_train_loss / len(train_loader):.5f}')

    # Validation
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        with tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs} - Validation', unit='batch') as t:
            for batch in t:
                val_input_data, _ = batch
                val_input_data = val_input_data.to(device)

                val_output_data = model(val_input_data)
                val_loss = criterion(val_output_data, val_input_data)
                total_val_loss += val_loss.item()
                t.set_postfix(loss=f'{total_val_loss / len(val_loader):.5f}')

    # Save model if validation loss is the lowest
    if total_val_loss < lowest_val_loss:
        lowest_val_loss = total_val_loss
        best_epoch = epoch + 1  # Epochs are 0-indexed, so add 1

        # Save the model's state dictionary and other relevant information
        checkpoint_path = 'best_model_checkpoint_haberman.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lowest_val_loss': lowest_val_loss,
        }, checkpoint_path)

print(f"Best model found at epoch {best_epoch} with validation loss: {lowest_val_loss:.5f}")


Epoch 1000/1000 - Validation: 100%|██████████| 2/2 [00:00<00:00, 329.86batch/s, loss=0.00313]

Best model found at epoch 666 with validation loss: 0.00172


In [8]:
checkpoint_path = 'best_model_checkpoint_haberman.pth'
checkpoint = torch.load(checkpoint_path)
model = Autoencoder(inputs_dim, n_bottleneck).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [9]:
tensor_data=torch.FloatTensor(df.values).to(device)
df_encoded=model.encoder(tensor_data)
df_encoded_cpu = df_encoded.cpu().detach().numpy()

In [10]:
import umap.umap_ as umap
df_umapped = umap.UMAP(n_components = n_bottleneck / 2, 
                           metric = "euclidean",
                           n_neighbors = 50, 
                           min_dist = 0.0,
                           random_state = 13).fit_transform(df_encoded_cpu)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [11]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Assuming you have your review_umapped data
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(df_umapped)

# Silhouette Score
silhouette_avg = silhouette_score(df_umapped, cluster_labels)
print(f'Silhouette Score: {silhouette_avg}')

# Davies-Bouldin Index
db_index = davies_bouldin_score(df_umapped, cluster_labels)
print(f'Davies-Bouldin Index: {db_index}')

# Calinski-Harabasz Index
ch_index = calinski_harabasz_score(df_umapped, cluster_labels)
print(f'Calinski-Harabasz Index: {ch_index}')


Silhouette Score: 0.5130324959754944
Davies-Bouldin Index: 0.6916902388543207
Calinski-Harabasz Index: 554.8388584555775


c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
